In [0]:
def pre_schema(path):
    try:
        df=spark.read.format("delta").load(f"{path}").limit(1)
        schema=""
        for i in df.dtypes:
            schema=schema+i[0]+" "+i[1]+","
        return schema[0:-1]
    except Exception as e:
        print("Error in pre_schema: ",str(e))


In [0]:
def f_delta_cleansed_load(table_name, location, database_name):
    try:
        schema=pre_schema(location)
        query = """
            CREATE TABLE IF NOT EXISTS hive_metastore.{3}.{0} ({2}) 
            USING DELTA 
            LOCATION '{1}'
        """.format(table_name, location, schema, database_name)
        spark.sql(query)
    except Exception as e:
        print("Error in f_delta_cleansed_load: ", str(e))


In [0]:
%py
def f_count_check(database, operation_type, table_name, number_diff):
    # Create a temporary view of the table's history
    spark.sql(f"DESC HISTORY {database}.{table_name}").createOrReplaceTempView("Table_count")
    
    # Get the current count of rows for the latest operation of the specified type
    count_current = spark.sql(f"""
        SELECT operationMetrics.numOutputRows 
        FROM Table_count 
        WHERE version = (
            SELECT MAX(version) 
            FROM Table_count 
            WHERE TRIM(LOWER(operation)) = LOWER('{operation_type}')
        )
    """)
    
    # Determine the final current count
    if count_current.first() is None:
        final_count_current = 0
    else:
        final_count_current = int(count_current.first().numOutputRows)
    
    # Get the previous count of rows before the latest operation of the specified type
    count_previous = spark.sql(f"""
        SELECT operationMetrics.numOutputRows 
        FROM Table_count 
        WHERE version < (
            SELECT version 
            FROM Table_count 
            WHERE LOWER(TRIM(operation)) = LOWER('{operation_type}') 
            ORDER BY version DESC 
            LIMIT 1
        )
    """)
    
    # Determine the final previous count
    if count_previous.first() is None:
        final_count_previous = 0
    else:
        final_count_previous = int(count_previous.first().numOutputRows or 0)
    
    # Check if the difference between current and previous counts exceeds the threshold
    if (final_count_current - final_count_previous) > number_diff:
        raise Exception(f"Difference is huge in {table_name} {final_count_current - final_count_previous}")
    else:
        pass